In [1]:
import pandas      as      pd
import numpy       as      np
import geopandas   as      gpd
import sys
from   itertools   import  product
import os
import datetime
import xarray      as xr

In [2]:
# inputs
path_org = '/home/shg096/scratch/test/gistool_outputs/comb/'
save_path = '/home/shg096/scratch/test/'

In [3]:
# load the combination for GeoData
Combinations  = xr.open_dataset('/home/shg096/scratch/test/gistool_outputs/comb/modified_West_Combination.nc')
fraction = pd.DataFrame(Combinations ['fraction'].values, index = Combinations['id'], columns = Combinations['comb_int'])
fraction = fraction.sort_index()
fraction.columns = ['SLC_' + str(col) for col in fraction.columns]
fraction

,SLC_1,SLC_2,SLC_3,SLC_4,SLC_5,SLC_6
28070338,0.000000,0.257438,0.310657,0.000000,0.103043,0.124345
28070572,0.000000,0.000000,0.000000,0.397431,0.408851,0.000000
28072028,0.000000,0.679530,0.000000,0.000000,0.164905,0.000000
28072168,0.000000,0.598935,0.218570,0.000000,0.000000,0.000000
28072213,0.309852,0.143802,0.000000,0.373161,0.173185,0.000000


In [4]:
# load network topology for GeoData
ntopo = xr.open_dataset('/home/shg096/scratch/test/domain/domain_ntopo.nc')
ntopo = ntopo.to_dataframe()
rename = {'COMID': 'subid', 'NextDownID': 'maindown', 'unitarea': 'area' , 'latitude': 'latitude' , 'longitude': 'longitude' , 'slope': 'slope_mean', 'length': 'rivlen', 'uparea': 'uparea'}
ntopo = ntopo[list(rename.keys())]
ntopo = ntopo.rename(columns=rename)
ntopo = ntopo.set_index('subid')
ntopo = ntopo.sort_index()
ntopo

,maindown,area,latitude,longitude,slope_mean,rivlen,uparea
subid,,,,,,,
28070338,-9999,2.156842e+08,37.581635,55.904027,0.014053,42455.019361,4.591212e+08
28070572,28070338,2.004575e+07,37.570312,55.993050,0.014008,7475.639047,1.562064e+08
28072028,28070572,8.214695e+07,37.466830,55.935032,0.029068,16550.179377,8.214695e+07
28072168,28070572,5.401367e+07,37.516098,55.894182,0.016343,10531.463128,5.401367e+07
28072213,28070338,8.723063e+07,37.510805,55.996791,0.017039,6534.799639,8.723063e+07


In [5]:
# get the elevation
elev = xr.open_dataset('/home/shg096/scratch/test/gistool_outputs/merit_hydro/domain_stats_elv.nc')
elev = elev.to_dataframe()
elev = elev.reset_index()
print(elev)
rename = {'COMID': 'subid', 'mean': 'elev_mean'}
elev = elev[list(rename.keys())]
elev = elev.rename(columns=rename)
elev = elev.set_index('subid')
elev = elev.sort_index()
elev

      COMID          min          max         mean       median
0  28070338   324.000000  1375.400024   874.133972   916.576361
1  28070572   916.700012  1293.300049  1060.109009  1048.332942
2  28072028  1025.599976  2160.699951  1598.861694  1674.067345
3  28072168  1025.500000  2030.200073  1369.633545  1272.675189
4  28072213   916.799988  1859.400024  1227.105103  1177.151851


,elev_mean
subid,
28070338,874.133972
28070572,1060.109009
28072028,1598.861694
28072168,1369.633545
28072213,1227.105103


In [12]:
# merge to create the GeoData
GeoClass = pd.concat([fraction, ntopo, elev], axis=1)
GeoClass['subid'] = GeoClass.index
GeoClass = GeoClass.sort_values(by = ['uparea'], ascending=True)
GeoClass = GeoClass.drop(columns = ['uparea'])
sorted_columns = ['subid', 'maindown', 'area', 'latitude', 'longitude', 'elev_mean', 'slope_mean', 'rivlen'] + \
[col for col in GeoClass.columns if col.startswith('SLC')]
GeoClass = GeoClass[sorted_columns]
GeoClass.to_csv(save_path+'/HYPE/GeoData.txt', sep='\t', index=False)
GeoClass

,subid,maindown,area,latitude,longitude,elev_mean,slope_mean,rivlen,SLC_1,SLC_2,SLC_3,SLC_4,SLC_5,SLC_6
28072168,28072168,28070572,5.401367e+07,37.516098,55.894182,1369.633545,0.016343,10531.463128,0.000000,0.598935,0.218570,0.000000,0.000000,0.000000
28072028,28072028,28070572,8.214695e+07,37.466830,55.935032,1598.861694,0.029068,16550.179377,0.000000,0.679530,0.000000,0.000000,0.164905,0.000000
28072213,28072213,28070338,8.723063e+07,37.510805,55.996791,1227.105103,0.017039,6534.799639,0.309852,0.143802,0.000000,0.373161,0.173185,0.000000
28070572,28070572,28070338,2.004575e+07,37.570312,55.993050,1060.109009,0.014008,7475.639047,0.000000,0.000000,0.000000,0.397431,0.408851,0.000000
28070338,28070338,-9999,2.156842e+08,37.581635,55.904027,874.133972,0.014053,42455.019361,0.000000,0.257438,0.310657,0.000000,0.103043,0.124345
